In [1]:
%pip install datasets transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", wh

In [2]:
import sklearn
import pandas as pd
import numpy as np
import random
import torch

seed = 1995

np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

Getting the dataset.

In [3]:
from datasets import load_dataset
sem_eval_2018_task_1 = load_dataset('sem_eval_2018_task_1', 'subtask5.english', trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

In [ ]:
%pip install deep-translator --quiet

In [ ]:
from collections import Counter
from deep_translator import GoogleTranslator

# Convert to DataFrame
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Shuffle the DataFrame
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)

# Select half of the DataFrame
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Define label columns
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
label_counts = {label: half_df[label].sum() for label in label_columns}
print("Current Label Counts:", label_counts)

# Function to perform back translation
def back_translate(text, src_lang='en', mid_lang='fr'):
    translated_text = GoogleTranslator(source=src_lang, target=mid_lang).translate(text)
    back_translated_text = GoogleTranslator(source=mid_lang, target=src_lang).translate(translated_text)
    return back_translated_text

# Augment tweets
augmented_data = []
processed_tweets = set()  # To track processed tweets

for index, row in half_df.iterrows():
    tweet = row['Tweet']
    original_labels = row[label_columns].to_dict()

    # Skip if this tweet has already been processed
    if index in processed_tweets:
        continue

    # Perform back translation
    new_tweet = back_translate(tweet)

    # Append augmented data
    augmented_data.append({'Tweet': new_tweet, **original_labels})

    # Update processed tweets set
    processed_tweets.add(index)

print(f"Number of tweets augmented: {len(augmented_data)}")

# Combine original and augmented datasets
augmented_df = pd.DataFrame(augmented_data)
combined_df = pd.concat([half_df, augmented_df], ignore_index=True)

# Verify label counts
new_label_counts = {label: combined_df[label].sum() for label in label_columns}
print("New Label Counts:", new_label_counts)

# Save combined dataset to CSV
combined_df.to_csv('augmented_data_seed2.csv', index=False)

Current Label Counts: {'anticipation': 473, 'optimism': 980, 'trust': 170, 'joy': 1249, 'love': 364, 'anger': 1270, 'disgust': 1299, 'pessimism': 397, 'sadness': 992, 'fear': 651, 'surprise': 166}
Number of tweets augmented: 3419
New Label Counts: {'anticipation': 946, 'optimism': 1960, 'trust': 340, 'joy': 2498, 'love': 728, 'anger': 2540, 'disgust': 2598, 'pessimism': 794, 'sadness': 1984, 'fear': 1302, 'surprise': 332}


BERT + 1/2 gold standard data.

In [ ]:
# Load your dataset into a DataFrame
df = pd.DataFrame(sem_eval_2018_task_1['train'])

label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Shuffle the DataFrame
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)

# Select half of the DataFrame
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Verify label distribution in the original and selected halves (optional)
original_label_distribution = df[label_columns].sum() / len(df)
selected_label_distribution = half_df[label_columns].sum() / len(half_df)

print("Original Label Distribution:")
print(original_label_distribution)
print("\nSelected Half Label Distribution:")
print(selected_label_distribution)

# Now `df_half` contains half of the original DataFrame with a label distribution close to the original

Original Label Distribution:
anticipation    0.143024
optimism        0.290143
trust           0.052208
joy             0.362240
love            0.102369
anger           0.372039
disgust         0.380521
pessimism       0.116262
sadness         0.293653
fear            0.181632
surprise        0.052793
dtype: float64

Selected Half Label Distribution:
anticipation    0.138345
optimism        0.286634
trust           0.049722
joy             0.365311
love            0.106464
anger           0.371454
disgust         0.379936
pessimism       0.116116
sadness         0.290143
fear            0.190407
surprise        0.048552
dtype: float64


In [ ]:
print(len(half_df))

3419


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(half_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(half_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5141, Val Loss: 0.4257
Epoch 2, Train Loss: 0.4050, Val Loss: 0.3732
Epoch 3, Train Loss: 0.3593, Val Loss: 0.3560
Epoch 4, Train Loss: 0.3242, Val Loss: 0.3356
Epoch 5, Train Loss: 0.2976, Val Loss: 0.3284
Test Accuracy: 25.87
Test F1-macro: 40.80
Test F1-micro: 65.42
Confusion Matrix:
Label: anticipation
[[2834    0]
 [ 425    0]]

Label: optimism
[[1745  371]
 [ 266  877]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1589  228]
 [ 272 1170]]

Label: love
[[2736    7]
 [ 476   40]]

Label: anger
[[1989  169]
 [ 315  786]]

Label: disgust
[[1925  235]
 [ 333  766]]

Label: pessimism
[[2881    3]
 [ 373    2]]

Label: sadness
[[2107  192]
 [ 421  539]]

Label: fear
[[2725   49]
 [ 230  255]]

Label: surprise
[[3089    0]
 [ 170    0]]



BERT + complete gold standard data

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4838, Val Loss: 0.3877
Epoch 2, Train Loss: 0.3692, Val Loss: 0.3432
Epoch 3, Train Loss: 0.3188, Val Loss: 0.3240
Epoch 4, Train Loss: 0.2879, Val Loss: 0.3084
Epoch 5, Train Loss: 0.2649, Val Loss: 0.3086
Test Accuracy: 28.47
Test F1-macro: 47.41
Test F1-micro: 68.82
Confusion Matrix:
Label: anticipation
[[2813   21]
 [ 405   20]]

Label: optimism
[[1809  307]
 [ 291  852]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1626  191]
 [ 268 1174]]

Label: love
[[2677   66]
 [ 311  205]]

Label: anger
[[1925  233]
 [ 243  858]]

Label: disgust
[[1855  305]
 [ 253  846]]

Label: pessimism
[[2864   20]
 [ 350   25]]

Label: sadness
[[2167  132]
 [ 408  552]]

Label: fear
[[2729   45]
 [ 196  289]]

Label: surprise
[[3089    0]
 [ 170    0]]



BERT + 1/2 gold standard data + 1/2 synthetic data

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("augmented_data_seed2.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data (assuming df_val remains unchanged)
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set (assuming df_test remains unchanged)
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4717, Val Loss: 0.3748
Epoch 2, Train Loss: 0.3477, Val Loss: 0.3306
Epoch 3, Train Loss: 0.2931, Val Loss: 0.3200
Epoch 4, Train Loss: 0.2590, Val Loss: 0.3218
Epoch 5, Train Loss: 0.2328, Val Loss: 0.3202
Test Accuracy: 26.60
Test F1-macro: 47.80
Test F1-micro: 67.21
Confusion Matrix:
Label: anticipation
[[2829    5]
 [ 417    8]]

Label: optimism
[[1890  226]
 [ 461  682]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1676  141]
 [ 306 1136]]

Label: love
[[2645   98]
 [ 279  237]]

Label: anger
[[1971  187]
 [ 293  808]]

Label: disgust
[[1910  250]
 [ 337  762]]

Label: pessimism
[[2802   82]
 [ 302   73]]

Label: sadness
[[2026  273]
 [ 312  648]]

Label: fear
[[2706   68]
 [ 183  302]]

Label: surprise
[[3089    0]
 [ 170    0]]



#RoBERTa

RoBERTa + half GSD

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10 # try increasing epochs
batch_size = 32
learning_rate = 1e-5 #5e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4 # Adjust this based on your memory constraints and desired effective batch size

# Load dataset (assuming combined_df and sem_eval_2018_task_1 are defined)
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(half_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(half_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.6127, Val Loss: 0.4966
Epoch 2, Train Loss: 0.4834, Val Loss: 0.4710
Epoch 3, Train Loss: 0.4567, Val Loss: 0.4114
Epoch 4, Train Loss: 0.4171, Val Loss: 0.3797
Epoch 5, Train Loss: 0.3943, Val Loss: 0.3629
Epoch 6, Train Loss: 0.3725, Val Loss: 0.3533
Epoch 7, Train Loss: 0.3622, Val Loss: 0.3432
Epoch 8, Train Loss: 0.3528, Val Loss: 0.3361
Epoch 9, Train Loss: 0.3382, Val Loss: 0.3303
Epoch 10, Train Loss: 0.3287, Val Loss: 0.3253
Test Accuracy: 26.88
Test F1-macro: 43.40
Test F1-micro: 66.64
Confusion Matrix:
Label: anticipation
[[2834    0]
 [ 425    0]]

Label: optimism
[[1754  362]
 [ 261  882]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1594  223]
 [ 278 1164]]

Label: love
[[2689   54]
 [ 340  176]]

Label: anger
[[1945  213]
 [ 276  825]]

Label: disgust
[[1895  265]
 [ 277  822]]

Label: pessimism
[[2882    2]
 [ 369    6]]

Label: sadness
[[2140  159]
 [ 445  515]]

Label: fear
[[2725   49]
 [ 280  205]]

Label: surprise
[[3089    0]
 [ 17

RoBERTa + original df

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10 # try increasing epochs
batch_size = 32
learning_rate = 1e-5 #5e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4 # Adjust this based on your memory constraints and desired effective batch size

# Load dataset (assuming combined_df and sem_eval_2018_task_1 are defined)
df = pd.DataFrame(sem_eval_2018_task_1['train'])
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5530, Val Loss: 0.4678
Epoch 2, Train Loss: 0.4295, Val Loss: 0.3661
Epoch 3, Train Loss: 0.3809, Val Loss: 0.3439
Epoch 4, Train Loss: 0.3514, Val Loss: 0.3299
Epoch 5, Train Loss: 0.3337, Val Loss: 0.3216
Epoch 6, Train Loss: 0.3218, Val Loss: 0.3115
Epoch 7, Train Loss: 0.3101, Val Loss: 0.3081
Epoch 8, Train Loss: 0.2991, Val Loss: 0.3048
Epoch 9, Train Loss: 0.2919, Val Loss: 0.2998
Epoch 10, Train Loss: 0.2859, Val Loss: 0.3003
Test Accuracy: 29.40
Test F1-macro: 52.09
Test F1-micro: 70.85
Confusion Matrix:
Label: anticipation
[[2789   45]
 [ 380   45]]

Label: optimism
[[1721  395]
 [ 222  921]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1628  189]
 [ 256 1186]]

Label: love
[[2538  205]
 [ 170  346]]

Label: anger
[[1966  192]
 [ 262  839]]

Label: disgust
[[1910  250]
 [ 277  822]]

Label: pessimism
[[2793   91]
 [ 285   90]]

Label: sadness
[[2118  181]
 [ 327  633]]

Label: fear
[[2678   96]
 [ 148  337]]

Label: surprise
[[3089    0]
 [ 17

RoBERTa + half GSD + half synthetic data

In [4]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10 # try increasing epochs
batch_size = 32
learning_rate = 1e-5 #5e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4 # Adjust this based on your memory constraints and desired effective batch size

# Load dataset (assuming combined_df and sem_eval_2018_task_1 are defined)
df = pd.read_csv("augmented_data_seed2.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5480, Val Loss: 0.4732
Epoch 2, Train Loss: 0.4487, Val Loss: 0.3834
Epoch 3, Train Loss: 0.3891, Val Loss: 0.3498
Epoch 4, Train Loss: 0.3539, Val Loss: 0.3382
Epoch 5, Train Loss: 0.3328, Val Loss: 0.3256
Epoch 6, Train Loss: 0.3146, Val Loss: 0.3160
Epoch 7, Train Loss: 0.2991, Val Loss: 0.3142
Epoch 8, Train Loss: 0.2873, Val Loss: 0.3140
Epoch 9, Train Loss: 0.2786, Val Loss: 0.3104
Epoch 10, Train Loss: 0.2698, Val Loss: 0.3105
Test Accuracy: 28.26
Test F1-macro: 51.18
Test F1-micro: 69.43
Confusion Matrix:
Label: anticipation
[[2792   42]
 [ 386   39]]

Label: optimism
[[1827  289]
 [ 341  802]]

Label: trust
[[3105    1]
 [ 153    0]]

Label: joy
[[1605  212]
 [ 265 1177]]

Label: love
[[2587  156]
 [ 210  306]]

Label: anger
[[1962  196]
 [ 288  813]]

Label: disgust
[[1910  250]
 [ 295  804]]

Label: pessimism
[[2770  114]
 [ 271  104]]

Label: sadness
[[2028  271]
 [ 284  676]]

Label: fear
[[2691   83]
 [ 163  322]]

Label: surprise
[[3088    1]
 [ 17